# Learning Goals
In Assignment 2, we learnt how to construct networks of spiking neurons and propagate information through a network of fixed weights. In this assignment, you will learn how to train network weights for a given task using brain-inspired learning rules.

Let's import all the libraries required for this assignment. 

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt

# Question 1: Training a Network

## 1a. 
What is the purpose of a learning algorithm? In other words, what does a learning algorithm dictate, and what is the objective of it?

## Answer 1a. 

An algorithm is nothing but a set of rules that are executed in a certain pattern. But when an algorithm has to make a prediction based on dataset , there aren't any concrete laid out rules that guarentees good evaluating performance. Thus there is a strong need for algorithms that can learn form the data and improve its performance on a particular task . The learning algorithm dictates its own rules which are sometimes very hard to come up mathematically. They recognize patterns and make predictions or decisions based on that learning. The ultimate objective is to perform a given task with flying colors(good evaluation)

## 1b. 
Categorize and explain the various learning algorithms w.r.t. biological plausibility. Can you explain the tradeoffs involved with the different learning rules? *Hint: Think computational advantages and disadvantages of biological plausibility.*

## Answer 1b. 

The general machine learning algorithms are mainly classified into 3 types. 

1. supervised learning - Training is based on taking output into consideration 
2. Unsupervised learning - output labels are not taken into consideration while  building models
3. Reinforcement learning - training based on a reward based system

But in case of biological plausibility : 
1. unsupervised algorithm has some plausibility here. below we have used correlation among the firing rates of neurons to update weights of our model. 
2. Hebbian learning : based on synaptic plasticity, which is the ability to change the strength based on the firing activity between them. (Idea : Neurons that wire together fire together). This has been shown to be  biologically plausible.
3. Reinforcement learning : It helps in learning a new activity based on rewards, which inturn is biologically plausible  


# Question 2: Hebbian Learning

## 2a.

In this exercise, you will implement the hebbian learning rule to solve AND Gate. First, we need to create a helper function to generate the training data. The function should return lists of tuples where each tuple comprises of numpy arrays of rate-coded inputs and the corresponding rate-coded output. 

Below is the function to generate the training data. Fill the components to return the training data. 

In [3]:
def genANDTrainData(snn_timestep):
    """ 
    Function to generate the training data for AND 
        Args:
            snn_timestep (int): timesteps for SNN simulation
        Return:
            train_data (list): list of tuples where each tuple comprises of numpy arrays of rate-coded inputs and output
        
        Write the expressions for encoding 0 and 1. Then append all 4 cases of AND gate to the list train_data
    """
    
    #Initialize an empty list for train data
    train_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
    
    #encode 1. Numpy random choice function might be useful here. 
    one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    # e.g., [0,0], [1,0], [0,1], [1,1]
    case_1 = [np.stack([zero,zero]),zero]
    case_2 = [np.stack([zero,one]),zero]
    case_3 = [np.stack([one,zero]),zero]
    case_4 = [np.stack([one,one]),one] 

    train_data.append(case_1)
    train_data.append(case_2)
    train_data.append(case_3)
    train_data.append(case_4)

    return train_data

## 2b. 
We will use the implementation of the network from assignment 2 to create an SNN comprising of one input layer and one output layer. Can you explain algorithmically, how you can use this simple architecture to learn AND gate. Your algorithm should comprise of encoding, forward propagation, network training, and decoding steps. 

## Answer 2b. 

SNN Architecture:
We have input layer and output layer with no hidden layers. With the input dimension being 2 and the output dimension being 1 . We use the firing rate(encoded values[0,1]) spikes as input to train our model.

Algorithm:
1. Build a SNN with vdecay and vth parameters
2. Intialize random weights for the connection between input and output
3. We iterate over the training data and computee the mean firing rate(output spikes) 
3. find the correlation between the input and the output spike and we will use this as heuristic to update the weights 
4. We run the model for the number of epochs mentioned

5. we decode the spikes using the threshold value
6. we test our trained model with new dataset 



The SNN has already been implemented for you. You do not need to do anything here. Just understand the implementation so that you can use it in the later parts. 

In [4]:
class LIFNeurons:
    """ 
        Define Leaky Integrate-and-Fire Neuron Layer 
        This class is complete. You do not need to do anything here.
    """

    def __init__(self, dimension, vdecay, vth):
        """
        Args:
            dimension (int): Number of LIF neurons in the layer
            vdecay (float): voltage decay of LIF neurons
            vth (float): voltage threshold of LIF neurons
        
        """
        self.dimension = dimension
        self.vdecay = vdecay
        self.vth = vth

        # Initialize LIF neuron states
        self.volt = np.zeros(self.dimension)
        self.spike = np.zeros(self.dimension)
    
    def __call__(self, psp_input):
        """
        Args:
            psp_input (ndarray): synaptic inputs 
        Return:
            self.spike: output spikes from the layer
                """
        self.volt = self.vdecay * self.volt * (1. - self.spike) + psp_input
        self.spike = (self.volt > self.vth).astype(float)
        return self.spike

class Connections:
    """ Define connections between spiking neuron layers """

    def __init__(self, weights, pre_dimension, post_dimension):
        """
        Args:
            weights (ndarray): connection weights
            pre_dimension (int): dimension for pre-synaptic neurons
            post_dimension (int): dimension for post-synaptic neurons
        """
        self.weights = weights
        self.pre_dimension = pre_dimension
        self.post_dimension = post_dimension
    
    def __call__(self, spike_input):
        """
        Args:
            spike_input (ndarray): spikes generated by the pre-synaptic neurons
        Return:
            psp: postsynaptic layer activations
        """
        psp = np.matmul(self.weights, spike_input)
        return psp
    
    
class SNN:
    """ Define a Spiking Neural Network with No Hidden Layer """

    def __init__(self, input_2_output_weight, 
                 input_dimension=2, output_dimension=2,
                 vdecay=0.5, vth=0.5, snn_timestep=20):
        """
        Args:
            input_2_hidden_weight (ndarray): weights for connection between input and hidden layer
            hidden_2_output_weight (ndarray): weights for connection between hidden and output layer
            input_dimension (int): input dimension
            hidden_dimension (int): hidden_dimension
            output_dimension (int): output_dimension
            vdecay (float): voltage decay of LIF neuron
            vth (float): voltage threshold of LIF neuron
            snn_timestep (int): number of timesteps for inference
        """
        self.snn_timestep = snn_timestep
        self.output_layer = LIFNeurons(output_dimension, vdecay, vth)
        self.input_2_output_connection = Connections(input_2_output_weight, input_dimension, output_dimension)
    
    def __call__(self, spike_encoding):
        """
        Args:
            spike_encoding (ndarray): spike encoding of input
        Return:
            spike outputs of the network
        """
        spike_output = np.zeros(self.output_layer.dimension)
        for tt in range(self.snn_timestep):
            input_2_output_psp = self.input_2_output_connection(spike_encoding[:, tt])
            output_spikes = self.output_layer(input_2_output_psp)
            spike_output += output_spikes
        return spike_output/self.snn_timestep      

## 2c. 
Next, you need to write a function for network training using hebbian learning rule. The function is defined below. You need to fill in the components so that the network weights are updated in the right manner. 

In [5]:
def hebbian(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        w = np.zeros((1,2))
        #iterate over all samples in train_data
        for data in train_data:
            #compute the firing rate for the input
            r1 = np.sum(data[0][0])/len(data[0][0])
            r2 = np.sum(data[0][1])/len(data[0][1])
            #compute the firing rate for the output
            ro = np.sum(data[1])/len(data[1])
            #compute the correlation using the firing rates calculated above
            cor1 = r1 * ro
            cor2 = r2 * ro
            #compute the weight increment
            w[0][0] = lr * cor1
            w[0][1] = lr * cor2
            #increment the weight
            network.input_2_output_connection.weights += w

## 2d. 
In this exercise, you will use your implementations above to train an SNN to learn AND gate. 

In [6]:
#Define a variable for input dimension
input_dim = 2

#Define a variable for output dimension
out_dim = 1

#Define a variable for voltage decay
vdecay=0.3

#Define a variable for voltage threshold
vth=0.2

#Define a variable for snn timesteps
snn_timestep=10

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(out_dim, input_dim) / 10

#print the initial weights
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_1 = SNN(input_2_output_weight, input_dim, out_dim, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)

#Train the network using the function defined in 2c. with the appropriate arguments
hebbian(network_1, train_data, lr=2e-2, epochs=20)

#Test the trained network and print the network output for all 4 cases. 
print("Testing ")


zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_1(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_1(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_1(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_1(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)

#Print Final Network Weights
print("Final Network Weights ", network_1.input_2_output_connection.weights)

initial weights  [[0.07197761 0.07401448]]
Testing 
zero  [0 0 0 1 0 0 0 1 1 0]
one  [1 1 0 1 1 0 1 1 0 0]
case 1 - output  [0.3]
case 2 - output  [0.7]
case 3 - output  [0.7]
case 4 - output  [0.6]
Final Output [case 1, case 2, case 3, case 4]  [1, 1, 1, 1]
Final Network Weights  [[0.43997761 0.44201448]]


# Question 3: Limitations of Hebbian Learning rule

## 3a. 
Can you learn the AND gate using 2 neurons in the output layer instead of one? If yes, describe what changes you might need to make to your algorithm in 2b. If not, explain why not, and what consequences it might entail for the use of hebbian learning for complex real-world tasks. 

## Answer 3a. 
Yes , we can definitely learn the and gate using 2 neurons in the output layer instead of one.

Changes :
1. intialize new weights to match the input and output dimensiom
2. We have to compute both the output values
3. The method for updating the weights has to be modified to handle both the outputs 
4. We might have to use negative weights andwe can encode 0 as -0.5 and 1 as +0 to consider the negative weights. 

Real world tasks:
We can use this for some real world tasks but it is going to be computationally expensive 

## 3b. 
Train the network using hebbian learning for AND gate with the same arguments as defined in 2d. but now multiply the number of epochs by 20. Can your network still learn AND gate correctly? Inspect the initial and final network weights, and compare them against the network weights in 2d. Based on this, explain your observations for the network behavior. 

In [7]:
#Implementation for 3b. (same as 2d. but with change of one argument)


input_2_output_weight = np.random.rand(out_dim, input_dim) / 10

print("initial weights ", input_2_output_weight)
network_2 = SNN(input_2_output_weight, input_dim, out_dim, vdecay, vth, snn_timestep)

hebbian(network_2, train_data, lr=2e-2, epochs=400)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_2(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_2(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_2(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_2(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_2.input_2_output_connection.weights)


initial weights  [[0.05479536 0.01582677]]
Testing 
zero  [0 0 0 0 0 1 0 0 1 1]
one  [1 1 1 1 0 0 1 1 0 1]
case 1 - output  [0.3]
case 2 - output  [0.9]
case 3 - output  [0.9]
case 4 - output  [0.7]
Final Output [case 1, case 2, case 3, case 4]  [1, 1, 1, 1]
Final Network Weights  [[7.41479536 7.37582677]]


## Answer 3b. 
 The network though looks like it has produced a better results here(Made 2 correct predictions) there is a strong possibility of over saturating the weights missing the optimal values. 
 
 The previous weights in 2b are "[0.20424399 0.25561099]" and the new weights are "[3.52568767 3.59409284]", which is large.

## 3c. 
Based on your observations and response in 3b., can you explain another limitation of hebbian learning rule w.r.t. weight growth? Can you also suggest a possible remedy for it?

## Answer 3c. 

The one main limitation of the hebbian learning technique is that it can lead to unbounded weight growth which causes the network to become unstable leading to numerical overflow and underflow.

Remedy of this problem would be to introduce normalization to limit the growth of the weights . some kind of regualarization term needs to be added to keep the weights in check

## 3d. 
To resolve the issues with hebbian learning, one possibility is Oja's rule. In this exercise, you will implement and train an SNN using Oja's learning rule. 

References: https://neuronaldynamics.epfl.ch/online/Ch19.S2.html

In [8]:
def oja(network, train_data, lr=1e-5, epochs=10):
    """ 
    Function to train a network using Hebbian learning rule
        Args:
            network (SNN): SNN network object
            train_data (list): training data 
            lr (float): learning rate
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples. 
        
        Write the operations required to compute the weight increment according to the hebbian learning rule. Then increment the network weights. 
    """
    
    #iterate over the epochs
    for ee in range(epochs):
        w = np.zeros((1,2))
        #iterate over all samples in train_data
        for data in train_data:
            #compute the firing rate for the input
            r1 = np.sum(data[0][0])/len(data[0][0])
            r2 = np.sum(data[0][1])/len(data[0][1])
            #compute the firing rate for the output
            ro = np.sum(data[1])/len(data[1])
            #compute the weight increment
            cor1 = r1 * ro
            cor2 = r2 * ro
            oja_term1 = network.input_2_output_connection.weights[0][0] * ro * ro
            oja_term2 = network.input_2_output_connection.weights[0][1] * ro * ro
            w[0][0] = lr * (cor1 - oja_term1)
            w[0][1] = lr * (cor2 - oja_term2)
            #increment the weight
            network.input_2_output_connection.weights += w

Now, test your implementation below. 

In [9]:
#Define a variable for input dimension
input_dim = 2
#Define a variable for output dimension
out_dim = 1
#Define a variable for voltage decay
vdecay = 0.5
#Define a variable for voltage threshold
vth = 0.5
#Define a variable for snn timesteps
snn_timestep = 5
#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(out_dim, input_dim) / 10
#print the initial weights
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_3 = SNN(input_2_output_weight, input_dim, out_dim, vdecay, vth, snn_timestep)
#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)
#Train the network using the function defined in 3d. with the appropriate arguments
oja(network_3, train_data, lr=2e-2, epochs=10)

#Test the trained network and print the network output for all 4 cases. 
print("Testing ")
zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_3(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_3(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_3(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_3(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
#Print Final Network Weights
print("Final Network Weights ", network_3.input_2_output_connection.weights)

initial weights  [[0.05324653 0.03960131]]
Testing 
zero  [0 0 0 1 0]
one  [0 1 1 1 1]
case 1 - output  [0.]
case 2 - output  [0.]
case 3 - output  [0.]
case 4 - output  [0.2]
Final Output [case 1, case 2, case 3, case 4]  [0, 0, 0, 0]
Final Network Weights  [[0.15535794 0.14295292]]


# Question 4: Spike-time dependent plasticity (STDP)

Reference: https://neuronaldynamics.epfl.ch/online/Ch19.S5.html

## 4a. 
What is the limitation with hebbian learning that STDP aims to resolve?


## Answer 4a. 

hebbain learning is unable to account for the temporal order of pre and post synaptic activity.
It also relies on correlation between the pre and post synaptic activity for updating the weights which leads to unbounded weight growth

In contrast, STDP takes into account the precise timing of pre- and post-synaptic spikes. STDP adjusts the strength of synapses based on the relative timing of pre- and post-synaptic spikes, with synapses that contribute to the postsynaptic response shortly after the presynaptic spike being strengthened, while those that contribute later being weakened. This allows for the formation of temporal associations between neurons and can enable the encoding of information in the precise timing of neural activity.

## 4b. 
Describe the algorithm to train a network using STDP learning rule. You do not need to describe encoding here. Your algorithm should be such that its naturally translatable to a program. 

## Answer 4b. 

In STDP learning, the weight updation is based on the temporal order of the pre and post synaptic spikes. when the pre-synaptic spike occures immediately before the post-synaptic spike , weight change is +VE(positive). In the other cases the weight change is negative.

The connectection between the two neurons in the network are associated with a time units delay. This time difference between pre and post-synaptic firing starts rising Since the model is envisioned to be used in digital systems, time is counted in discrete units. The membrane potential is described as a function of time and is increased by a synaptic weight value for each incoming spike. A constant value is subtracted from the membrane potential at every time instant to take into account the delay of time units. When the membrane potential crosses the threshold potential, the neuron produces a spike and the membrane potential decreases to the resting potential, which is the minimum potential level of a neuron.



## 4c. 
In this exercise, you will implement the STDP learning algorithm to train a network. STDP has many different flavors. For this exercise, we will use the learning rule defined in: https://dl.acm.org/doi/pdf/10.1609/aaai.v33i01.330110021. Pay special attention to Equations 2 and 3. 

Below is the class definition for STDP learning algorithm. Your task is to fill in the components so that the weights are updated in the right manner. 

In [12]:
class STDP():
    """Train a network using STDP learning rule"""
    def __init__(self, network, A_plus, A_minus, tau_plus, tau_minus, lr, snn_timesteps=20, epochs=30, w_min=0, w_max=1):
        """
        Args:
            network (SNN): network which needs to be trained
            A_plus (float): STDP hyperparameter
            A_minus (float): STDP hyperparameter
            tau_plus (float): STDP hyperparameter
            tau_minus (float): STDP hyperparameter
            lr (float): learning rate
            snn_timesteps (int): SNN simulation timesteps
            epochs (int): number of epochs to train with. Each epoch is defined as one pass over all training samples.  
            w_min (float): lower bound for the weights
            w_max (float): upper bound for the weights
        """
        self.network = network
        self.A_plus = A_plus
        self.A_minus = A_minus
        self.tau_plus = tau_plus
        self.tau_minus = tau_minus
        self.snn_timesteps = snn_timesteps
        self.lr = lr
        self.time = np.arange(0, self.snn_timesteps, 1)
        self.sliding_window = np.arange(-4, 4, 1) #defines a sliding window for STDP operation. 
        self.epochs = epochs
        self.w_min = w_min
        self.w_max = w_max
    
    def update_weights(self, t, i):
        """
        Function to update the network weights using STDP learning rule
        
        Args:
            t (int): time difference between postsynaptic spike and a presynaptic spike in a sliding window
            i(int): index of the presynaptic neuron
        
        Fill the details of STDP implementation
        """
        #compute delta_w for positive time difference
        if t>0:
            delta_w = self.A_plus * np.exp(-t / self.tau_plus)
        #compute delta_w for negative time difference
        else:
            delta_w = -self.A_minus * np.exp(-t / self.tau_minus)
            
        #update the network weights if weight increment is negative
        if delta_w < 0:
            change = self.lr * delta_w * (self.network.input_2_output_connection.weights - self.w_min)
            self.network.input_2_output_connection.weights += change 
        #update the network weights if weight increment is positive
        elif delta_w > 0:
            change = self.lr * delta_w * (self.w_max - self.network.input_2_output_connection.weights)
            self.network.input_2_output_connection.weights += change 
            
    def train_step(self, train_data_sample):
        """
        Function to train the network for one training sample using the update function defined above. 
        
        Args:
            train_data_sample (list): a sample from the training data
            
        This function is complete. You do not need to do anything here. 
        """
        input = train_data_sample[0]
        output = train_data_sample[1]
        for t in self.time:
            if output[t] == 1:
                for i in range(2):
                    for t1 in self.sliding_window:
                        if (0<= t + t1 < self.snn_timesteps) and (t1!=0) and (input[i][t+t1] == 1):
                            self.update_weights(t1, i)
    
    def train(self, training_data):
        """
        Function to train the network
        
        Args:
            training_data (list): training data
        
        This function is complete. You do not need to do anything here. 
        """
        for ee in range(self.epochs):
            for train_data_sample in training_data:
                self.train_step(train_data_sample)

Let's test the implementation

In [13]:
#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1
#Define a variable for voltage decay
vdecay = 0.5
#Define vdecay = 0.5
vth = 0.36
#Define a variable for snn timesteps
snn_timestep = 5
#Initialize randomly the weights from input to output. Numpy random rand function might be useful here. 
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10
print("initial weights ", input_2_output_weight)
#Initialize an snn using the arguments defined above
network_4 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)
#Get the training data for AND gate using the function defined in 2a. 
train_data = genANDTrainData(snn_timestep)
#Create an object of STDP class with appropriate arguments
STDP_obj = STDP(network_4, A_plus = 0.6, A_minus = 0.3, tau_plus = 8, tau_minus = 5, lr = 0.25, 
                snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
STDP_obj.train(train_data)
#Test the trained network and print the network output for all 4 cases. 
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_4(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_4(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_4(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_4(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)

#Print Final Network Weights
print("Final Network Weights ", network_4.input_2_output_connection.weights)

initial weights  [[0.04958204 0.09193069]]
Testing 
zero  [0 0 1 0 1]
one  [1 0 1 0 1]
case 1 - output  [0.4]
case 2 - output  [0.4]
case 3 - output  [0.4]
case 4 - output  [0.6]
Final Output [case 1, case 2, case 3, case 4]  [1, 1, 1, 1]
Final Network Weights  [[0.23895906 0.23895906]]


# Question 5: OR Gate
Can you train the network with the same architecture in Q2-4 for learning the OR gate. You will need to create another function called genORTrainData. Then create an SNN and train it using STDP. 

In [14]:
#Write your implementation of genORTrainData here. 

def genORTrainData(snn_timestep):
    
    #Initialize an empty list for train data
    training_data = []
    
    #encode 0. Numpy random choice function might be useful here. 
    zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
    
    #encode 1. Numpy random choice function might be useful here. 
    one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
    
    #Append all 4 cases of AND gate to train_data. Numpy stack operation might be useful here. 
    i1 = np.array([zero,zero])
    i2 = np.array([zero,one])
    i3 = np.array([one,zero])
    i4 = np.array([one,one])
    
    case_1 = np.array([i1,zero])
    case_2 = np.array([i2,one])
    case_3 = np.array([i3,one])
    case_4 = np.array([i4,one])
    training_data = np.stack((case_1, case_2, case_3, case_4))

    return training_data


In [15]:
#Train the network for OR gate here using the implementation from 4c. 


#Define a variable for input dimension
input_dimension = 2

#Define a variable for output dimension
output_dimension = 1

#Define a variable for voltage decay
vdecay = 0.5

#Define a variable for voltage threshold
vth = 0.39

#Define a variable for snn timesteps
snn_timestep = 5

#Initialize randomly the weights from input to output. Numpy random rand function might be useful here.
input_2_output_weight = np.random.rand(output_dimension, input_dimension) / 10
print("initial weights ", input_2_output_weight)

#Initialize an snn using the arguments defined above
network_5 = SNN(input_2_output_weight, input_dimension, output_dimension, vdecay, vth, snn_timestep)

#Get the training data for AND gate using the function defined in 2a. 
training_data = genORTrainData(snn_timestep)

#Create an object of STDP class with appropriate arguments
OR_obj = STDP(network_5, A_plus = 0.6, A_minus = 0.3, tau_plus = 8, tau_minus = 5, lr = 0.25, 
                snn_timesteps=5, epochs=30, w_min=0, w_max=1)

#Train the network using STDP
OR_obj.train(training_data)

#Test the trained network and print the network output for all 4 cases.
print("Testing ")

zero = np.random.choice([1,0], snn_timestep, p=[0.3, 0.7])
print("zero ", zero)

one = np.random.choice([1,0], snn_timestep, p=[0.7, 0.3])
print("one ", one)
    
i1 = np.array([zero,zero])
i2 = np.array([zero,one])
i3 = np.array([one,zero])
i4 = np.array([one,one])

output = []

#case 1
out1 = network_5(i1)
print("case 1 - output ", out1)
if out1 > vth:
    output.append(1)
else:
    output.append(0)

#case 2
out2 = network_5(i2)
print("case 2 - output ", out2)
if out2 > vth:
    output.append(1)
else:
    output.append(0)

#case 3
out3 = network_5(i3)
print("case 3 - output ", out3)
if out3 > vth:
    output.append(1)
else:
    output.append(0)

#case 4
out4 = network_5(i4)
print("case 4 - output ", out4)
if out4 > vth:
    output.append(1)
else:
    output.append(0)
print("Final Output [case 1, case 2, case 3, case 4] ", output)
    
#Print Final Network Weights
print("Final Network Weights ", network_5.input_2_output_connection.weights)

initial weights  [[0.00837704 0.03865927]]
Testing 
zero  [1 0 0 0 0]
one  [1 1 0 1 0]
case 1 - output  [0.2]
case 2 - output  [0.2]
case 3 - output  [0.2]
case 4 - output  [0.6]
Final Output [case 1, case 2, case 3, case 4]  [0, 0, 0, 1]
Final Network Weights  [[0.27701864 0.27701864]]


<ipython-input-14-359cb58b920a>:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  case_1 = np.array([i1,zero])
<ipython-input-14-359cb58b920a>:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  case_2 = np.array([i2,one])
<ipython-input-14-359cb58b920a>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  case_3 = np.array([i3,one])
<ip